#### Файл сервера xmlrpc_stats_server_p3.ipynb на Python 3 

In [11]:

from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler

import datetime
import os
import pandas as pd

import sqlite3

# Подключение к базе данных (если базы данных нет, она будет создана)
conn = sqlite3.connect('log.db')

# Создание объекта курсора для выполнения SQL-запросов
cursor = conn.cursor()

# Создание таблицы
cursor.execute('''
    CREATE TABLE IF NOT EXISTS server_log (
        id INTEGER PRIMARY KEY,
        action_name TEXT NOT NULL,
        date TEXT,
        duration INTEGER       
    )
''')

class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

stat_server = SimpleXMLRPCServer(("localhost", 8011), 
                            requestHandler=RequestHandler)

insert = "INSERT INTO server_log (action_name, date, duration) VALUES (?, ?, ?)"
select = 'SELECT * FROM server_log'

# Добавление строки в лог
def add_log(action_name, date, duration):
    cursor.execute(insert, (action_name, date, duration))
    conn.commit()
    return True

stat_server.register_function(add_log, 'add_log')

def get_log_df():
    df = pd.DataFrame()
    cursor.execute(select)
    rows = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(rows, columns=columns)
    return df.to_json()

stat_server.register_function(get_log_df, 'get_log_df')

print ("Listening on " + str(stat_server.server_address))

try: 
    stat_server.serve_forever()
    print('That was a nice nap.')
except KeyboardInterrupt:
    stat_server.server_close()
    conn.close()
    print('What a rude awakening!')


Listening on ('127.0.0.1', 8011)


127.0.0.1 - - [28/Nov/2023 00:25:12] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2023 00:25:14] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2023 00:25:51] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2023 00:25:53] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2023 00:26:21] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2023 00:26:23] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2023 00:26:59] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2023 00:27:01] "POST /RPC2 HTTP/1.1" 200 -


What a rude awakening!
